<a href="https://colab.research.google.com/github/ArtemDorofeev/IT-education-project/blob/main/Get_data_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vk_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import vk_api
import csv

ACCESS_TOKEN = 'vk1.a.pSKBMznrZEAj0QKshXGiJV538mjVDypdz2EOpjQohj6TgY-Gfjz0wvikMYl7ZzsOwtaAez11FVt_3ew5PhniaX6CqrZvfQzRhjLkSNaQsF7jmD0CSH9UaJpxHTAaXzu5XqVwElNWDRMnJXJ7sVlG6a8e2SsrMSLwTo5MzT8y6etUawpXP0YFBxJHMd4KxllT'

# Инициализация сессии с помощью токена доступа

vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()


## **Задание 1. Собираем данные с помощью Requests**

С помощью методов API ВКонтакте получите список высших учебных заведений и названия их факультетов в городе Томск.
Результат должен быть записан в файл JSON в следующем формате:

In [ ]:
# Получение ID города Томск
city = vk.database.getCities(country_id=1, q="Томск", count=1)
city_id = city["items"][0]["id"] if city["count"] > 0 else None
city_name = city["items"][0]["title"] if city["count"] > 0 else None

city_name

'Томск'

In [ ]:
# Получение списка университетов и факультетов

if city_id:
  univer_data = vk.database.getUniversities(country_id=1, city_id=city_id, count=1000)
  universities = []
  for i in univer_data['items']:
    facult_data = vk.database.getFaculties(university_id=i['id'])
    faculties = [fk['title'] for fk in facult_data['items']]
    university = {'id':i['id'], 'name':i['title'], 'faculties':faculties}
    universities.append(university)
else:
   print(f"ERROR: not city")
dic = {"cities": [{"id": city_id, "name": city_name, "universities": universities}]}


In [ ]:
# Запись данных в файл json

import json

with open('Tomsk_universities.json', 'w') as f:
    f.write(json.dumps(dic, ensure_ascii=False))


# **Задание 2. Библиотека VK API**

Напишите код, который получает список названия школ города Кемерово с помощью библиотеки vk_api и записывает результаты в файл JSON.

In [ ]:
# Получение ID города Кемерово

city = vk.database.getCities(country_id=1, q="Кемерово", count=1)
city_id = city["items"][0]["id"] if city["count"] > 0 else None
city_name = city["items"][0]["title"] if city["count"] > 0 else None

city_name

'Кемерово'

In [ ]:
# Получение списка школ

if city_id:
  school_data = vk.database.getSchools(city_id=city_id)
  schools = [shl['title'] for shl in school_data['items']]
else:
   print(f"ERROR: not city")

In [ ]:
# Запись данных в файл json

import json

with open('Kemerovo_schools.json', 'w') as f:
    f.write(json.dumps(schools, ensure_ascii=False))

# **Задание 3. Запись данных в CSV формат**

С помощью методов API «ВКонтакте» получите 1000 подписчиков группы «Лентач», отсортирванных по дате регистрации.

Вам необходимо собрать следующие данные в CSV файл: пол, название город, семейное положение (ФИО партнера не указывать).

In [ ]:
# Получение ID группы Лентач

group_data = vk.groups.search(q='Лентач')
group_id = group_data["items"][0]["id"] if group_data["count"] > 0 else None


In [ ]:
# Получение списка подписчиков

if group_id:
  members_data = vk.groups.getMembers(group_id=group_id, sort='id_asc', fields=['sex', 'city', 'relation'], count=1000)
else:
   print(f"ERROR: not group")


In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame(members_data['items'])

In [ ]:
# Приводим значение "Пол"

df['sex_a'] = df['sex'].apply(lambda x: 'жен' if int(x) == 1 else 'муж')

In [ ]:
# Приводим значение "Город"

def city_nan(x):
  try:
    city = x['title']
    return city
  except:
    city = 'не указано'
    return city

df['city_a'] = df['city'].apply(city_nan)

In [ ]:
# преобразование кодов семейного статуса в текстовое значение

def relations(status):
  if status == 1:
    n = 'не женат/не замужем'
  elif status == 2:
    n = 'есть друг/есть подруга'
  elif status == 3:
    n = 'помолвлен/помолвлена'
  elif status == 4:
    n = 'женат/замужем'
  elif status == 5:
    n = 'всё сложно'
  elif status == 6:
    n = 'в активном поиске'
  elif status == 7:
    n = 'влюблён/влюблена'
  elif status == 8:
    n = 'в гражданском браке'
  else:
    n = 'не указано'
  return n


In [ ]:
# Приводим значение "Отношения"

def relation_nan(x):
  try:
    rel = relations(int(x))
    return rel
  except:
    rel = 'не указано'
    return rel

df['relation_a'] = df['relation'].apply(relation_nan)

In [ ]:
df[['sex_a', 'city_a', 'relation_a']].to_csv('Lentach_1000_members.csv', index=False, header=['Пол', 'Город', 'Cемейное положение'])